# Data Wrangling With MongoDB Project

**Project 2 from Udacity Data Analyst Nano Degree program              by Jay Zmudka**

---

In [47]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET
from pprint import pprint as pp
import re
import codecs
import json
from zipfile import ZipFile


In [34]:
samplefile = 'sample.osm'
zipfile = 'openstreetmapdata.zip'
filename = 'openstreetmapdata'
outfile = 'openstreetmapdata.json'

# uncomment the data you are using smaller sample file or larger zipped file
usefile = samplefile
#usefile = openzipfile(zipfile, filename)

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# extracts osm file from zip archive
def openzipfile(zipf, filename):
    with ZipFile(zipf) as osmzip:
        xfile = osmzip.open(filename)
        return xfile

        


In [35]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        node['id'] = element.attrib['id']
        if 'visible' in element.attrib:
            node['visible'] = element.attrib['visible']
        node['created'] = {}
        for x in CREATED:
            node['created'][x] = element.attrib[x]
        if 'lat' in element.attrib:
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        if len(element.findall('tag')) > 0:
            node['address'] = {}
            for tag in element.findall('tag'):
#                   print tag.attrib
                
                if problemchars.search(tag.attrib['k']):
                    pass
                elif 'addr:' in tag.attrib['k']:
                    if len(tag.attrib['k'].split(':')) > 2:
                        pass
                    else:
                        addc = tag.attrib['k'].split(':')[1]
                        node['address'][addc] = tag.attrib['v']
                else:
                    node[tag.attrib['k']] = tag.attrib['v']
        if len(element.findall('nd')) > 0:
            node['node_refs'] = []
            for nd in element.findall('nd'):
                node['node_refs'].append(nd.attrib['ref'])

        
        return node
    else:
        return None

    

In [48]:
# read data into elementree objects



data = []

for _, element in ET.iterparse(usefile):
    piece = shape_element(element)
    if piece != None:
        data.append(piece)

pp(data[:10])



[{'address': {},
  'created': {'changeset': '56806337',
              'timestamp': '2018-03-02T03:10:43Z',
              'uid': '1084189',
              'user': 'lmum',
              'version': '5'},
  'id': '18834600',
  'pos': [42.3140859, -83.0758705],
  'source': 'PGS',
  'type': 'node',
  'visible': 'true'},
 {'address': {},
  'created': {'changeset': '11542302',
              'timestamp': '2012-05-08T18:51:24Z',
              'uid': '583595',
              'user': 'a_white',
              'version': '4'},
  'id': '18834601',
  'pos': [42.3148796, -83.0746638],
  'source': 'PGS',
  'type': 'node',
  'visible': 'true'},
 {'address': {},
  'created': {'changeset': '65690620',
              'timestamp': '2018-12-22T11:13:47Z',
              'uid': '6656962',
              'user': 'StudentinGear',
              'version': '4'},
  'id': '18834604',
  'pos': [42.3155278, -83.0737495],
  'source': 'PGS',
  'type': 'node',
  'visible': 'true'},
 {'address': {},
  'created': {'changeset': 